In [1]:
from typing import Annotated, List, Dict, Any
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

In [2]:
class ChatState(TypedDict):
    messages: Annotated[List, add_messages]

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")
def chatbot_node(state: ChatState) -> Dict[str, Any]:
    messages = state["messages"]
    ai = llm.invoke(messages)
    return {"messages": [ai]}

In [4]:
graph = StateGraph(ChatState)
graph.add_node("chatbot", chatbot_node)

graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

In [5]:
def test(user_input, thread_id):
    config = {"configurable": {"thread_id": thread_id}}
    state: ChatState = {
        "messages": [HumanMessage(content=user_input)]
    }
    state = app.invoke(state, config=config)
    for m in state['messages']:
        print(m.__class__.__name__, ":", m.content)

In [6]:
test("Hi My name is Manmath", "demo-001")

HumanMessage : Hi My name is Manmath
AIMessage : Hi Manmath! How can I assist you today?


In [7]:
test("What is my name?", "demo-001")

HumanMessage : Hi My name is Manmath
AIMessage : Hi Manmath! How can I assist you today?
HumanMessage : What is my name?
AIMessage : Your name is Manmath. How can I assist you further?
